This notebook follows the basic tutorial series of Krish Nayek for training and testing a ML model in AWS sagemaker. The code has been updated to solve the errors provided in the tutorials.

# Steps to be followed

* Import necessary libraries
* Creating S3 Bucket
* Mapping train and test data in S3
* Mapping the path of the models in S3
* Download and load the datasets
* Load the datasets into the S3  bucket
* Load the sagemaker built-in model
* Train the model
* Create trained model endpoints 
* Test the model
* Delete everything after testing

In [48]:
import sagemaker
import boto3 #used for accessing s3 bucket
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import image_uris
from sagemaker.session import s3_input, Session

In [33]:
bucket_name = "ak-bankapp001"
my_region = boto3.session.Session().region_name
print(my_region)

eu-north-1


***Create a S3 bucket***

In [34]:
s3 = boto3.resource('s3')
try:
    if(my_region == 'eu-north-1'):
        s3.create_bucket(Bucket=bucket_name,CreateBucketConfiguration={'LocationConstraint': 'eu-north-1'})
    print("S3 bucket created successfully")
except Exception as e:
    print('S3 error: ', e)

S3 bucket created successfully


***Set output path for the model to be saved while being trained***

In [35]:
prefix = "xgboost-as-a-built-in-algo"
output_path = 's3://{}/{}/output'.format(bucket_name,prefix)
print(output_path)

s3://ak-bankapp001/xgboost-as-a-built-in-algo/output


***Downloading the dataset and store it in the bucket***

In [37]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv",'bank_clean.csv')
    print("Suceeeded in downloading the bank_data.csv")
except Exception as e:
    print("data load error: ", e)

Suceeeded in downloading the bank_data.csv


In [39]:
try:
    model_data = pd.read_csv("./bank_clean.csv", index_col=0)
    print("Suceess: data loaded into dataframe")
except exception as e:
    print("data loader error: ", e)

Suceess: data loaded into dataframe


Train_test_split

In [40]:
import numpy as np
train_data, test_data = np.split(model_data.sample(frac= 1, random_state= 175), [int(0.7* len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


***Save Train data into the bucket***

For sagemaker, the dependent column should be the first feature. So the dependent colum should be placed first and the other independent columns should be concatenated after it


In [42]:
import os
pd.concat([train_data["y_yes"], train_data.drop(['y_no','y_yes'], axis=1)],
         axis=1).to_csv('train.csv', index=False, header= False)

In [43]:
#upload the train csv in the bucket
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train= sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name,prefix), content_type='csv')

In [44]:
pd.concat([test_data["y_yes"], test_data.drop(['y_no','y_yes'], axis=1)],
         axis=1).to_csv('test.csv', index=False, header= False)

In [57]:
#upload the test csv in the bucket
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test= sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name,prefix), content_type='csv')

***Building model XGBOOST***

uses the in-bulit sagemaker xgboost algorithm. It is presented in the form of a container or an image. 

In [51]:
container = image_uris.retrieve(region= boto3.Session().region_name,
                                framework='xgboost',
                                version= '1.0-1')

In [59]:
# initialize hyperparameters
hyperparameters = {
    "max_depth": "5",
    "num_round": "100",
    "eta": "0.2",
    "gamma" : "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "objective": "binary:logistic",
}


Construct a Sagemaker estimator that calls the XGBOOST-container

In [60]:
estimator = sagemaker.estimator.Estimator(image_uri= container,
                                         hyperparameters=hyperparameters,
                                         role=sagemaker.get_execution_role(),
                                         instance_count=1,
                                         instance_type = 'ml.m5.2xlarge',
                                         volume_size = 5, #5 GB
                                         output_path= output_path,
                                         use_spot_instances= True,
                                         max_run= 300,
                                         max_wait=600)

In [61]:
estimator.fit({'train': s3_input_train, 'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-07-03-05-53-49-356


2023-07-03 05:53:49 Starting - Starting the training job...
2023-07-03 05:54:03 Starting - Preparing the instances for training......
2023-07-03 05:54:59 Downloading - Downloading input data...
2023-07-03 05:55:40 Training - Training image download completed. Training in progress...[2023-07-03 05:56:03.329 ip-10-0-203-124.eu-north-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node 

***Deploy The ML Model as Endpoints***

In [63]:
xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-07-03-06-06-57-097
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2023-07-03-06-06-57-097
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2023-07-03-06-06-57-097


----!

 ***Prediction of the test Data***
    
When we are giving data to the endpoints, the input is usually in the form of an excel dataset/ tabular dataset. This needs to be serialized before feeding it to the model end point. 

In [67]:
from sagemaker.serializers import CSVSerializer

In [72]:
# Create a predictor
predictor = sagemaker.predictor.Predictor("sagemaker-xgboost-2023-07-03-06-06-57-097")

In [73]:
# Set the content type and serializer type
predictor.content_type = 'text/csv'
predictor.serializer = CSVSerializer()

# Define the test data
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array

# Make predictions
predictions = predictor.predict(test_data_array).decode('utf-8')

# Process the prediction result
predictions_array = np.fromstring(predictions[1:], sep=',')

# Print the shape of the predictions array
print(predictions_array.shape)

(12357,)


In [74]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 90.6%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10830)    27% (138)
Purchase        9% (1019)     73% (370) 



***Delete the endpoints***

We have to delete the endpoints and the buckets in order to stop the billing.

In [75]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2023-07-03-06-06-57-097


[{'ResponseMetadata': {'RequestId': '51CBE32BD16CRJ47',
   'HostId': 'qCr4QE18kAyEzDrTRPljZyGFGX9v6EUNCQOq5II3i/sa2Rk2Ste9NqH9qPUul+BPQqKE2O1/vVg=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'qCr4QE18kAyEzDrTRPljZyGFGX9v6EUNCQOq5II3i/sa2Rk2Ste9NqH9qPUul+BPQqKE2O1/vVg=',
    'x-amz-request-id': '51CBE32BD16CRJ47',
    'date': 'Mon, 03 Jul 2023 06:52:46 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-07-03-05-53-49-356/profiler-output/system/incremental/2023070305/1688363700.algo-1.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-07-03-05-53-49-356/debug-output/index/000000000/000000000090_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-07-03-05-42-49-216/debug-output/training_job_end.ts'},
   {'Key': 'xgboost-as-a-bui